# Photon Transfer Notebook

Analyze ramp data, creating linearity and photon transfer curves.

In [1]:
# Make sure that any changes to the dependencies are automatically updated
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
# Tools for opening image stacks and relevant data from fits headers
from cosmos_helper_funcs import get_stacks, label_plot, cosmos_gain_dict, get_mean_images
# Tools for making defect pixel histograms
from defect_pixel_funcs import *
from scipy.optimize import curve_fit

In [ ]:
# The folder in which a PTC ramp is stored
ramp_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/MKI Lab data/10-11-2024/Mask Ramps/HSHG_RS/Light'
# Load the files containing the image stacks for the ramp. For each stack, extract only the mean image and the standard deviation image
ramp_images = get_stacks(ramp_folder, get_mean_img=True, get_var_img=False, num_imgs=0, datatype='int')

In [ ]:
# The folder in which a PTC ramp is stored
dark_folder = '/Users/layden/Library/CloudStorage/Box-Box/Scientific CMOS - MKI ONLY (might contain EAR; ITAR)/Teledyne_COSMOS/MKI Lab data/10-11-2024/Mask Ramps/HSHG_RS/Dark'
# Load the files containing the image stacks for the ramp. For each stack, extract only the mean image and the standard deviation image
dark_images = get_stacks(dark_folder, get_mean_img=True, get_var_img=False, num_imgs=0, datatype='int')

In [7]:
# Sort bias_stacks by exposure time
ramp_images = sorted(ramp_images, key=lambda x: float(x["exposure_ms"]))
dark_images = sorted(dark_images, key=lambda x: float(x["exposure_ms"]))
exp_times_light = np.array([float(x["exposure_ms"]) for x in ramp_images])
exp_times_dark = np.array([float(x["exposure_ms"]) for x in dark_images])
for i, frame in enumerate(ramp_images):
    frame["mean_img"] = frame["mean_img"] - dark_images[i]["mean_img"]

In [8]:
row_100_vals = np.array([x["mean_img"][90:110, :].mean(axis=0) for x in ramp_images])

In [ ]:
%matplotlib qt
exp_times_array = np.zeros((9, len(exp_times_light)))
illumination_factors = [1, 1.0133, 1.0338, 1.067, 1, 1, 1, 1, 1]
for i in range(9):
    illumination_factor = row_100_vals[70, 0] / row_100_vals[70, i * 1000]
    print(illumination_factor)
    exp_times_array[i] = exp_times_light / illumination_factors[i]
    plt.scatter(exp_times_array[i], row_100_vals[:, i * 1000], s=2, label=f'Row 100, Col {i * 1000}')
plt.xlabel('Corrected Exposure time (ms)')
plt.ylabel('Mean pixel value (ADU)')
plt.legend()
plt.show()

In [ ]:
print(ramp_images[0]["mean_img"])